# Diffbind analysis to identify differential peaks between the different cell types

requires precomputed diffbind dataset

In [1]:
library("tidyverse")
library("DiffBind")
library("readxl")
library("pheatmap")
options(bitmapType="cairo")

setwd("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/ATAC_seq_analysis")


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.1.0
✔ tidyr   1.1.4     ✔ stringr 1.4.1
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, pa

In [2]:
dataset_info_file = "../metadata/cleaned_ATAC_metadata.csv"
dataset_info = read.csv(dataset_info_file)
dataset_peaks_location = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/master_ATAC_ChIP_analyzer/macs2"
dataset_alignment_location = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/master_ATAC_ChIP_analyzer/clean_alignments"

dataset_info <- dataset_info %>% filter(condition %in% c("healthy", "patient", "synovium"))

data_for_diffbind <- dataset_info %>% select(id, patient, cell_type, condition, active_disease, female_sex, on_bDMARD_tsDMARD,
on_csDMARD, on_steroid, On_MTX, group, age, disease_duration, cell_type) %>% mutate(folder = paste0(id, "_ATAC"))
data_for_diffbind <- data_for_diffbind %>% mutate(Peaks = paste0(dataset_peaks_location, "/", folder, "/", folder, "_peaks_nosex.narrowPeak"), 
bamReads = paste0(dataset_alignment_location, "/", folder, "/", folder, "_align_filtered_macs2.bam"), PeakCaller = "narrow")

load(".local/diffbind_object.Rdata")

data_object$class[DBA_TISSUE, ] = data_for_diffbind$cell_type
data_object$class[DBA_FACTOR, ] = data_for_diffbind$female_sex

norm_dba_object <- dba.normalize(data_object, normalize=DBA_NORM_NATIVE,
library=DBA_LIBSIZE_PEAKREADS,background=FALSE)

## running individual contrasts for the results we want

In [4]:
# Differences between CD8 and CD4 T cells

res_condition <- dba(norm_dba_object,data_for_diffbind$condition != "synovium")
res_condition = dba.contrast(res_condition, design = "~ Tissue + Factor",
    contrast = c("Tissue", "CD8", "CD4"))

res_condition = dba.analyze(res_condition)
differential_peaks <- dba.report(res_condition, th = 1)
write.csv(differential_peaks, file=".local/diffbind_result/DE_CD8_vs_CD4_all.csv")

Computing results names...

Normalize DESeq2 with defaults...

Analyzing...

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates



In [5]:
# Differences between CD4 SF with CD4
## including all

res_condition <- dba(norm_dba_object,data_for_diffbind$cell_type %in% c("CD4", "CD4_SF"))
res_condition = dba.contrast(res_condition, design = "~ Tissue + Factor",
    contrast = c("Tissue", "CD4_SF", "CD4"))

res_condition = dba.analyze(res_condition)
differential_peaks <- dba.report(res_condition, th = 1)
write.csv(differential_peaks, file=".local/diffbind_result/DE_CD4SF_vs_CD4_all.csv")

Computing results names...

Normalize DESeq2 with defaults...

Analyzing...

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates



In [6]:
## looking only at high disease activity

res_condition <- dba(norm_dba_object,data_for_diffbind$cell_type %in% c("CD4", "CD4_SF") & data_for_diffbind$active_disease == "1")
res_condition = dba.contrast(res_condition, design = "~ Tissue + Factor",
    contrast = c("Tissue", "CD4_SF", "CD4"))

res_condition = dba.analyze(res_condition)
differential_peaks <- dba.report(res_condition, th = 1)
write.csv(differential_peaks, file=".local/diffbind_result/DE_CD4SF_vs_CD4_high_activity.csv")

Computing results names...

Normalize DESeq2 with defaults...

Analyzing...

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates



In [7]:
# Differences between CD8 SF with CD8
## including all

res_condition <- dba(norm_dba_object,data_for_diffbind$cell_type %in% c("CD8", "CD8_SF"))
res_condition = dba.contrast(res_condition, design = "~ Tissue + Factor",
    contrast = c("Tissue", "CD8_SF", "CD8"))

res_condition = dba.analyze(res_condition)
differential_peaks <- dba.report(res_condition, th = 1)
write.csv(differential_peaks, file=".local/diffbind_result/DE_CD8SF_vs_CD8_all.csv")

Computing results names...

Normalize DESeq2 with defaults...

Analyzing...

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates



In [8]:
## looking only at high disease activity

res_condition <- dba(norm_dba_object,data_for_diffbind$cell_type %in% c("CD8", "CD8_SF") & data_for_diffbind$active_disease == "1")
res_condition = dba.contrast(res_condition, design = "~ Tissue + Factor",
    contrast = c("Tissue", "CD8_SF", "CD8"))

res_condition = dba.analyze(res_condition)
differential_peaks <- dba.report(res_condition, th = 1)
write.csv(differential_peaks, file=".local/diffbind_result/DE_CD8SF_vs_CD8_high_activity.csv")

Computing results names...

Normalize DESeq2 with defaults...

Analyzing...

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates



In [9]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Scientific Linux 7.4 (Nitrogen)

Matrix products: default
BLAS:   /opt/gridware/el7/apps/gcc/R/4.1.2/lib64/R/lib/libRblas.so
LAPACK: /opt/gridware/el7/apps/gcc/R/4.1.2/lib64/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] pheatmap_1.0.12             readxl_1.3.1               
 [3] DiffBind_3.4.7              SummarizedExperiment_1.24.0
 [5] Biobase_2.54.0              MatrixGenerics_1.6.0       
 [7] matrixStats_0.61.0          Genom